# Q2

In [14]:
import pandas as pd

df = pd.read_excel('PS4data.xls')

In [15]:
df

,year,quarter,real consumption of nondurables,real consumption of services,cpi,real disposable income,population,equally weighted average return on the New York Stock Exchange
0,1948,1,442.3,463.8,23.533333,1055.3,102690.67,0.01414
1,1948,2,446.9,470.5,23.933333,1087.7,102915.33,0.11755
2,1948,3,443.3,476.6,24.466667,1107.1,103249.00,-0.09565
3,1948,4,449.6,480.2,24.233333,1109.8,103417.67,-0.04563
4,1949,1,451.8,482.6,23.866667,1087.8,103584.33,0.03338
...,...,...,...,...,...,...,...,...
215,2001,4,1885.0,3616.6,177.266670,6729.1,216111.67,0.14591
216,2002,1,1921.4,3642.2,177.900000,6961.0,216664.00,0.08077
217,2002,2,1920.9,3666.2,179.833330,7027.2,217203.67,-0.03911
218,2002,3,1925.8,3687.0,180.600000,7058.1,217867.67,-0.16290


## (a)

In [46]:
import statsmodels.api as sm

df['cpc'] = (df['real consumption of nondurables'] + df['real consumption of services']) / df['population']
y = df['cpc']

p = 2  # number of lags
model = sm.tsa.AutoReg(y, lags=4, trend='c')
results = model.fit(cov_type='HC1')


In [47]:
print(results.summary())

                            AutoReg Model Results                             
Dep. Variable:                    cpc   No. Observations:                  220
Model:                     AutoReg(4)   Log Likelihood                1745.123
Method:               Conditional MLE   S.D. of innovations              0.000
Date:                Thu, 20 Nov 2025   AIC                          -3478.245
Time:                        14:59:07   BIC                          -3457.994
Sample:                             4   HQIC                         -3470.064
                                  220                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.439e-05   1.96e-05      1.242      0.214   -1.41e-05    6.29e-05
cpc.L1         1.1923      0.069     17.363      0.000       1.058       1.327
cpc.L2        -0.1281      0.103     -1.241      0.2

In [48]:
import statsmodels.api as sm

# H0: beta2 = beta3 = beta4
test = results.wald_test("cpc.L2 = cpc.L3 = cpc.L4 = 0")
test.summary()

/Users/andrewkaplan/miniconda3/envs/RA/lib/python3.11/site-packages/statsmodels/base/model.py:1912: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


'<Wald test (chi2): statistic=[[20.19210279]], p-value=0.00015487120491656955, df_denom=3>'

In [49]:
results.f_test("cpc.L2 = cpc.L3 = cpc.L4 = 0")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=6.730700931608538, p=0.00023377665765132125, df_denom=211, df_num=3>

reject the null

## (c)

In [19]:
df.head()

,year,quarter,real consumption of nondurables,real consumption of services,cpi,real disposable income,population,equally weighted average return on the New York Stock Exchange,cpc
0,1948,1,442.3,463.8,23.533333,1055.3,102690.67,0.01414,0.008824
1,1948,2,446.9,470.5,23.933333,1087.7,102915.33,0.11755,0.008914
2,1948,3,443.3,476.6,24.466667,1107.1,103249.00,-0.09565,0.008910
3,1948,4,449.6,480.2,24.233333,1109.8,103417.67,-0.04563,0.008991
4,1949,1,451.8,482.6,23.866667,1087.8,103584.33,0.03338,0.009021


In [20]:
import numpy as np

df_reg = pd.DataFrame()

df_reg['income'] = df['real disposable income'] / df['population']
df_reg['ichange'] = np.log(df_reg['income'] / df_reg['income'].shift(1))
df_reg['cchange'] = np.log(df['cpc'] / df['cpc'].shift(1))
df_reg['cchangel1'] = df_reg['cchange'].shift(2)
df_reg['cchangel2'] = df_reg['cchange'].shift(3)
df_reg['cchangel3'] = df_reg['cchange'].shift(4)
df_reg['cchangel4'] = df_reg['cchange'].shift(5)

In [ ]:
import statsmodels.api as sm
from linearmodels.iv import IV2SLS

# Drop missing observations
df_2sls = df_reg.dropna(subset=['cchange', 'ichange', 'cchangel1', 'cchangel2', 'cchangel3', 'cchangel4'])

# Define variables
y = df_2sls['cchange']
X = sm.add_constant(df_2sls['ichange'])
Z = sm.add_constant(df_2sls[['cchangel1', 'cchangel2', 'cchangel3', 'cchangel4']])

# 2SLS estimation
iv = IV2SLS(
    dependent=y,
    exog=sm.add_constant(pd.DataFrame(index=df_2sls.index)),  # constant term
    endog=df_2sls['ichange'],
    instruments=df_2sls[['cchangel1', 'cchangel2', 'cchangel3', 'cchangel4']]
)
res = iv.fit(cov_type='robust')
print(res.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                cchange   R-squared:                      0.0685
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0641
No. Observations:                 214   F-statistic:                    4.6529
Date:                Thu, Nov 20 2025   P-value (F-stat)                0.0310
Time:                        14:04:52   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0026     0.0012     2.2579     0.0240      0.0003      0.0049
ichange        0.4369     0.2026     2.1571     0.03

# (d)

In [ ]:
# Exogeneity tests

# non robust
res.wu_hausman()

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 3.3841
P-value: 0.0672
Distributed: F(1,211)
WaldTestStatistic, id: 0x14504f990

In [ ]:
# robust
res.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 1.1399
P-value: 0.2857
Distributed: chi2(1)
WaldTestStatistic, id: 0x1658d0990

In [31]:
# Overidentification test
res.sargan

Sargan's test of overidentification
H0: The model is not overidentified.
Statistic: 7.7383
P-value: 0.0517
Distributed: chi2(3)
WaldTestStatistic, id: 0x1044904d0

In [32]:
# Robust
print(res.wooldridge_overid)

Wooldridge's score test of overidentification
H0: Model is not overidentified.
Statistic: 5.0870
P-value: 0.1655
Distributed: chi2(3)


# 3

## (d)

In [51]:
df = pd.read_excel('SP500Index.xlsx')

In [53]:
df['indx'] = np.log(df['Level of the S&P 500 Index'] / df['Level of the S&P 500 Index'].iloc[0])

In [66]:
# delta

T = len(df) - 1
dh = df['indx'].iloc[-1]/T

dh

0.005548394684384916

In [71]:
#sigma 

df['indx_diff'] = df['indx'].diff()

x = (df['indx_diff'] - dh) **2
x.dropna(inplace=True)

np.sqrt(x.sum() / T)

0.04218034607391596